# Index Data
Purpose is to uplaod the data to the Azure AI Search Index
This uses the PUSH API, however it is also very viable to use the Azure AI Search Indexer that points to the JSON files directly

## Required for this step
- Azure AI Search Service

## Important
- Ensure that Semantic Ranker is enabled in Azure AI Search service



In [ ]:
# # Import required libraries  
import os  
import json
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient  


In [ ]:
#Load the configuration details for the Cognitive Search Service and Azure OpenAI Instance
#Credentials should be secured using a more secure method such as Azure KeyVault

from dotenv import load_dotenv
load_dotenv()

# Load configuration from environment variables
config = {
    "search_service_name": os.getenv("SEARCH_SERVICE_NAME"),
    "search_admin_key": os.getenv("SEARCH_ADMIN_KEY"),
    "search_index_name": os.getenv("SEARCH_INDEX_NAME"),
    "search_index_schema_file": os.getenv("SEARCH_INDEX_SCHEMA_FILE"),
    "search_api_version": os.getenv("SEARCH_API_VERSION", "2023-07-01"),
    "data_root_dir": os.getenv("DATA_ROOT_DIR", "./data")
}

    
# Azure AI Search Config
search_service_name = config["search_service_name"]
search_service_url = "https://{}.search.windows.net/".format(search_service_name)
search_admin_key = config["search_admin_key"]
index_name = config["search_index_name"]
index_schema_file = config["search_index_schema_file"]
search_api_version = config["search_api_version"]
search_headers = {  
    'Content-Type': 'application/json',  
    'api-key': search_admin_key  
}  

index_client = SearchIndexClient(
        endpoint=search_service_url, credential=AzureKeyCredential(search_admin_key))
search_client = SearchClient(endpoint=search_service_url, index_name=index_name, credential=AzureKeyCredential(search_admin_key))

print ('Search Service Name:', search_service_name)
print ('Index Name:', index_name)

# Set a temp directory for downloading pdf's for processing
data_root_dir = config["data_root_dir"]
json_dir = os.path.join(data_root_dir, "json")
print ('JSON Dir:', json_dir)

In [ ]:
# Get all files in dir
def get_files_in_dir(in_dir):
    return [os.path.join(dp, f) for dp, dn, filenames in os.walk(in_dir) for f in filenames]


In [ ]:
json_docs = get_files_in_dir(json_dir)
total_files = len(json_docs)
print ('Total Files to upload:', total_files)

In [ ]:
for json_doc in json_docs:
    print (json_doc)
    with open(json_doc, "r") as j_in:
        documents = json.loads(j_in.read())

    result = search_client.upload_documents(documents)  
    print(f"Uploaded {len(documents)} chunks") 